In [1]:

#Libraries

import tensorflow as tf
import os
from tensorflow import keras

import numpy as np
from matplotlib import pyplot as plt

import fastapi 
import imageio 
import cv2
import imghdr


#Librerias para el modelo
from sklearn.model_selection import RepeatedKFold, cross_val_score
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
from tensorflow.keras import layers, models # type: ignore
from tensorflow.keras.layers import Input   # type: ignore


In [2]:


gpus = tf.config.list_physical_devices('GPU')
print(gpus)


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:

#Preprocesamiento de imagenes 
potholes_dir = './pothole_image_data/Pothole_Image_Data'
road_dir ='./pothole_image_data/pp'

#etiqutas 
def load_images(folder,label):
    images=[]
    labels=[]
    for filename in os.listdir(folder):
        img_path = os.path.join(folder,filename)
        img= cv2.imread(img_path)
        img= cv2.resize(img,(512,512))
        img= img/255.0
        images.append(img)
        labels.append(label)
    return images,labels


pothole_img,pothole_lb = load_images(potholes_dir, 1) # 1 para baches

road_surface_img, road_surface_lb = load_images(road_dir, 0)  # 0 para carretera


# Combinar y dividir el dataset
X = np.array(pothole_img + road_surface_img)
y = np.array(pothole_lb + road_surface_lb)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Preprocesamiento para datos de entrenamiento
X_train.shape, X_test.shape, y_train.shape, y_test.shape    



((768, 512, 512, 3), (193, 512, 512, 3), (768,), (193,))

In [8]:
#Data augmentaton

#Flujo de imagenes 
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # type: ignore

train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    zoom_range=0.15,
    horizontal_flip=True

)

validation_datagen = ImageDataGenerator()


# Create a generator for validation data
validation_generator = validation_datagen.flow(X_test, y_test, batch_size=32)
#Generator for train data

train_generator =train_datagen.flow(X_train,y_train,batch_size=32)



In [ ]:

#Modelo base
#le fallta tuneo

#Creamos la CNN
model = models.Sequential([
    Input(shape=(256,256,3)),#Input layer
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Salida binaria
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [6]:
model.save('modelapi_shitty.h5')

In [10]:

#Entrenar y evaluar el modelo

history = model.fit(
    train_generator,
    steps_per_epoch=len(X_train) // 32,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=len(X_test) // 32
)

# Evaluación del modelo
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_accuracy:.2f}')


Epoch 1/10
24/24 [==============================] - 16s 287ms/step - loss: 0.8483 - accuracy: 0.6198 - val_loss: 0.5579 - val_accuracy: 0.7760
Epoch 2/10
24/24 [==============================] - 7s 284ms/step - loss: 0.4660 - accuracy: 0.7878 - val_loss: 0.3864 - val_accuracy: 0.8385
Epoch 3/10
24/24 [==============================] - 7s 295ms/step - loss: 0.4520 - accuracy: 0.8008 - val_loss: 0.4477 - val_accuracy: 0.8385
Epoch 4/10
24/24 [==============================] - 7s 306ms/step - loss: 0.4460 - accuracy: 0.8112 - val_loss: 0.3518 - val_accuracy: 0.8698
Epoch 5/10
24/24 [==============================] - 7s 299ms/step - loss: 0.4126 - accuracy: 0.8242 - val_loss: 0.3553 - val_accuracy: 0.8802
Epoch 6/10
24/24 [==============================] - 7s 297ms/step - loss: 0.3772 - accuracy: 0.8385 - val_loss: 0.4183 - val_accuracy: 0.8542
Epoch 7/10
24/24 [==============================] - 7s 302ms/step - loss: 0.3855 - accuracy: 0.8411 - val_loss: 0.4400 - val_accuracy: 0.8594
Epoch

In [25]:
import numpy as np
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import KFold, cross_val_score
from tensorflow.keras import models, layers, Input

def create_model():
    model = models.Sequential([
        Input(shape=(256, 256, 3)),  # Capa de entrada
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(1, activation='sigmoid')  # Salida binaria
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Preparar los datos (ejemplo)
X_train = np.random.rand(100, 256, 256, 3)
y_train = np.random.randint(2, size=100)

model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=32, verbose=0)
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

results = cross_val_score(model, X_train, y_train, cv=kfold)
print(f'Cross-Validation Accuracy Scores: {results}')
print(f'Mean Accuracy: {results.mean()}')
print(f'Standard Deviation: {results.std()}')

C:\Users\herre\AppData\Local\Temp\ipykernel_28000\523974004.py:26: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=32, verbose=0)


Cross-Validation Accuracy Scores: [0.40000001 0.60000002 0.60000002 0.5        0.60000002 0.40000001
 0.60000002 0.69999999 0.40000001 0.69999999]
Mean Accuracy: 0.5500000089406967
Standard Deviation: 0.11180339754218994
